In [8]:
from langchain.llms import GooglePalm

api_key = 'API_KEY' # get this free api key from https://makersuite.google.com/

llm = GooglePalm(google_api_key=api_key, temperature=0.9)

In [1]:
pip show langchain

Name: langchain
Version: 0.0.339
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: C:\Users\nihal\anaconda3\Lib\site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show torch


Name: torchNote: you may need to restart the kernel to use updated packages.

Version: 2.1.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: C:\Users\nihal\anaconda3\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, invisible-watermark, peft, sentence-transformers, torchaudio, torchvision


In [9]:
poem = llm("Write a 4 line funny story of Harry stealing Ron's wand.")
print(poem)

Harry crept into Ron's room,
"I'm borrowing this," he whispered,
"Just for a bit," he grinned,
And slunk away with Ron's wand.


In [10]:
essay = llm("Write a 10 line story where Charlie Chaplin drops a bottle of oil on fat man and the waiter throws a chicken on fat man by mistake.")
print(essay)

1. Charlie Chaplin is walking down the street when he sees a fat man eating a chicken.
2. Charlie Chaplin drops a bottle of oil on the fat man, who is covered in grease.
3. The waiter comes out of the restaurant and sees the fat man covered in grease.
4. The waiter thinks that the fat man is trying to steal a chicken, so he throws a chicken at him.
5. The chicken hits the fat man in the face and he falls to the ground.
6. Charlie Chaplin laughs and walks away, leaving the fat man covered in grease and chicken.
7. The fat man gets up and chases after Charlie Chaplin, but he is too slow.
8. Charlie Chaplin runs away laughing, and the fat man is left standing alone, covered in grease and chicken.
9. The fat man realizes that he has been tricked, and he starts to cry.
10. Charlie Chaplin laughs at the fat man, and then he walks away.


In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='jokes.csv', source_column="Question",encoding="utf8")

# Store the loaded data in the 'data' variable
data = loader.load()

In [13]:
data

[Document(page_content='ID: 1\nQuestion: Did you hear about the Native American man that drank 200 cups of tea?\nAnswer: He nearly drown in his own tea pee.', metadata={'source': 'Did you hear about the Native American man that drank 200 cups of tea?', 'row': 0}),
 Document(page_content="ID: 2\nQuestion: What's the best anti diarrheal prescription?\nAnswer: Mycheexarphlexin", metadata={'source': "What's the best anti diarrheal prescription?", 'row': 1}),
 Document(page_content='ID: 3\nQuestion: What do you call a person who is outside a door and has no arms nor legs?\nAnswer: Matt', metadata={'source': 'What do you call a person who is outside a door and has no arms nor legs?', 'row': 2}),
 Document(page_content='ID: 4\nQuestion: Which Star Trek character is a member of the magic circle?\nAnswer: Jean-Luc Pickacard', metadata={'source': 'Which Star Trek character is a member of the magic circle?', 'row': 3}),
 Document(page_content="ID: 5\nQuestion: What's the difference between a bull

In [14]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("Why did a chicken cross the road?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [16]:
rdocs = retriever.get_relevant_documents("Did you hear about the Native American man that drank 200 cups of tea?")
rdocs

[Document(page_content='ID: 1\nQuestion: Did you hear about the Native American man that drank 200 cups of tea?\nAnswer: He nearly drown in his own tea pee.', metadata={'source': 'Did you hear about the Native American man that drank 200 cups of tea?', 'row': 0}),
 Document(page_content='ID: 128\nQuestion: Did you hear about that Native American who drank ten cups of tea one night?\nAnswer: They found him dead the next day in his teepee', metadata={'source': 'Did you hear about that Native American who drank ten cups of tea one night?', 'row': 127}),
 Document(page_content='ID: 518\nQuestion: What do Englishmen like more than tea?\nAnswer: Tea tea', metadata={'source': 'What do Englishmen like more than tea?', 'row': 517}),
 Document(page_content='ID: 631\nQuestion: How does a Jewish man make coffee?\nAnswer: Hebrews it', metadata={'source': 'How does a Jewish man make coffee?', 'row': 630})]

In [72]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs={"prompt": PROMPT})

In [19]:
chain("Why is bear so popular in north pole?")

{'query': 'Why is bear so popular in north pole?',
 'result': 'there is a lot of white',
 'source_documents': [Document(page_content='ID: 824\nQuestion: How does water stay warm during the winter?\nAnswer: It wears an aqua-fur.', metadata={'source': 'How does water stay warm during the winter?', 'row': 823}),
  Document(page_content="ID: 663\nQuestion: Why did Frosty drop his pants?\nAnswer: Because he's a pervert that likes showing people his snowballs.", metadata={'source': 'Why did Frosty drop his pants?', 'row': 662}),
  Document(page_content='ID: 129\nQuestion: What do you call a bear with no teeth?\nAnswer: Bare Grylls', metadata={'source': 'What do you call a bear with no teeth?', 'row': 128}),
  Document(page_content='ID: 905\nQuestion: Why is the Kremlin so much warmer this winter?\nAnswer: It had new windows Putin', metadata={'source': 'Why is the Kremlin so much warmer this winter?', 'row': 904})]}

In [71]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate a funny answer based on this context only.
In the answer try to provide as much text as possible from "answer" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "No idea. Hehe." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT=PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
)



In [76]:
chain("Why bears are popular in north pole?")

{'query': 'Why bears are popular in north pole?',
 'result': "Answer: ... They're the only ones that can stand the cold.",
 'source_documents': [Document(page_content='ID: 824\nQuestion: How does water stay warm during the winter?\nAnswer: It wears an aqua-fur.', metadata={'source': 'How does water stay warm during the winter?', 'row': 823}),
  Document(page_content="ID: 663\nQuestion: Why did Frosty drop his pants?\nAnswer: Because he's a pervert that likes showing people his snowballs.", metadata={'source': 'Why did Frosty drop his pants?', 'row': 662}),
  Document(page_content='ID: 378\nQuestion: Why do gorillas have big nostrils?\nAnswer: ... Because they got big fingers.', metadata={'source': 'Why do gorillas have big nostrils?', 'row': 377}),
  Document(page_content='ID: 129\nQuestion: What do you call a bear with no teeth?\nAnswer: Bare Grylls', metadata={'source': 'What do you call a bear with no teeth?', 'row': 128})]}